In [1]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
11,application_1659447865675_0004,pyspark,idle,Link,Link


SparkSession available as 'spark'.
Connected. Call `.close()` to terminate connection gracefully.

In [ ]:
# Preserve original send_request method
from hsfs.client import base
send_request_original = base.Client._send_request

In [48]:
def wrap_send_request(response_instance):

    def _send_request_wrap(
            self,
            method,
            path_params,
            query_params=None,
            headers=None,
            data=None,
            stream=False,
            files=None,
    ):
        global send_request_original
        response = send_request_original(self, method, path_params, query_params, headers, data, stream, files)
        response_instance.response = response
        response_instance.method = method
        response_instance.path_params = path_params
        response_instance.query_params = query_params
        response_instance.headers = headers
        response_instance.data = data
        response_instance.stream = stream
        response_instance.files = files
        return response

    hsfs.client.base.Client._send_request = _send_request_wrap
    
def unwrap_send_request():
    global send_request_original
    base.Client._send_request = send_request_original

In [49]:
class RequestResponseInstance:
    def __init__(self):
        self.response = None
        self.method = None
        self.path_params = None
        self.query_params = None
        self.headers = None
        self.data = None
        self.stream = None
        self.files = None
        
    def to_dict(self):
        d = {}
        d["response"] = self.response
        d["method"] = self.method
        d["path_params"] = self.path_params
        d["query_params"] = self.query_params
        d["headers"] = self.headers
        return d

class ResponseGenerator:
    
    def __init__(self, name):
        self.name = name

    def prepare(self):
        pass

    def call(self):
        pass

    def cleanup(self):
        pass
    
    def run(self):
        global responses_dict
        if self.name in responses_dict:
            raise Exception("fixture was already determined. remove instance from responses_dict or rename generator to continue.")
        
        response_instance = RequestResponseInstance()

        self.prepare()
        
        wrap_send_request(response_instance)
        self.call()
        unwrap_send_request()
        
        self.cleanup()
        
        responses_dict[self.name] = response_instance.to_dict()
        
        return response_instance

In [50]:
# specify generators...

class FeatureGroupResponseGenerator(ResponseGenerator):

    def prepare(self):

        from pyspark.sql.types import StructType, StructField, StringType, IntegerType
        data2 = [(1, "asd"),(2, "asssd"),(23, "adssd"),(1, "adsasd"),(7, "asds")]

        schema = StructType([
            StructField("intt",IntegerType(),True),
            StructField("stringt",StringType(),True)
        ])

        df = spark.createDataFrame(data=data2,schema=schema)

        from hsfs.feature import Feature
        features = [
            Feature(name="intt",type="int",online_type="int"),
            Feature(name="arrt",type="array<int>",online_type="varchar(1000)")
        ]

        features = [
            Feature(name="intt",type="int",online_type="int"),
            Feature(name="stringt",type="string",online_type="varchar(1000)")
        ]
        self.fg = fs.create_feature_group(name="fg_test",
                                     features=features,
                                     primary_key=["intt"], # key can not contain null values
                                     online_enabled=True,
                                     time_travel_format="HUDI")

        
        self.fg.save(df)

    def call(self):
        fs.get_feature_group("fg_test", version=1)
        
    def cleanup(self):
        self.fg.delete()

In [51]:
responses_dict = {}

# run generators...
FeatureGroupResponseGenerator("get_feature_group").run()    

Feature Group created successfully, explore it at 
https://hopsworks0.logicalclocks.com/p/119/fs/67/fg/1050

In [45]:
# write responses captured with the generators to 'backend_fixtures.json'
import json
import pydoop.hdfs as hdfs

filename = f'hdfs:///Projects/{fs.project_name}/Resources/backend_fixtures.json'
with hdfs.open(filename, 'wt') as json_file:
    json.dump(responses_dict, json_file, 
                        indent=4,  
                        separators=(',',': '))